In [42]:
import json
import requests
from datetime import datetime, timedelta
import pytz
from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

In [44]:
a = datetime.utcnow() - signer.utcnow
a < timedelta(minutes=5)


False

In [80]:
class Reviews():
    def __init__(self, **kwargs):
        
        self.auth_headers = {
            'Accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://www.goodreads.com/',
            'X-Amz-User-Agent': 'aws-sdk-js/2.711.0 callback',
            'Content-Type': 'application/x-amz-json-1.1',
            'X-Amz-Target': 'AWSCognitoIdentityService.GetCredentialsForIdentity',
            'X-Amz-Content-Sha256': '6030ca7c1f3393246848c0323a5e26fea5b3f98543b92a2e409fdf030aba5245',
            'Origin': 'https://www.goodreads.com',
            'Connection': 'keep-alive',
        }
        self.auth_url = "https://cognito-identity.us-east-1.amazonaws.com/"
        self.auth_payload = "{\"IdentityId\":\"us-east-1:5cb98804-4a0e-4004-aeae-f8feb41b7be5\"}"
        
        self.method = "POST"
        self.host = "kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com"
        self.region = "us-east-1"
        self.service = "appsync" 
        self.url = "https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql"
        
        self.get_auth()
        self.make_review_payload(**kwargs)
        
    def get_auth(self):
         # get credentials
        self.utcnow = datetime.utcnow()
        self.amz_date = self.utcnow.strftime("%Y%m%d")
        self.amz_time = self.utcnow.strftime("%H%M%S")
        # now = datetime.datetime.now(pytz.timezone('US/Eastern'))

        response = requests.request(
            self.method,
            self.auth_url,
            headers=self.auth_headers,
            data=self.auth_payload
        )
        self.auth = json.loads(response.text)["Credentials"]
        
        # write to credentials, which is read by default for env variables
        with open(os.path.expanduser("~/.aws/credentials"), "w") as f_out:
            f_out.write(
                "[default]"
                f"\naws_access_key_id={signer.auth['AccessKeyId']}"
                f"\naws_secret_access_key={signer.auth['SecretKey']}"
                f"\naws_session_token={signer.auth['SessionToken']}"
            )
            
    def check_auth(self):
        if (datetime.utcnow() - self.utcnow) >= timedelta(seconds=300):
            self.get_auth()
            
    def make_review_payload(self, review_num=64):
        self.review_num = review_num
        # not worth prettifying this string, but it does set review_num pagination
        self.reviews_payload = "{\"operationName\":\"getReviews\",\"variables\":{\"filters\":{\"resourceType\":\"WORK\",\"resourceId\":\"kca://work/amzn1.gr.work.v1.jY9lyLQNtOx-St1AGeHgBA\"},\"pagination\":{\"limit\":"+str(self.review_num)+"}},\"query\":\"query getReviews($filters: BookReviewsFilterInput!, $pagination: PaginationInput) {\\n  getReviews(filters: $filters, pagination: $pagination) {\\n    ...BookReviewsFragment\\n    __typename\\n  }\\n}\\n\\nfragment BookReviewsFragment on BookReviewsConnection {\\n  totalCount\\n  edges {\\n    node {\\n      ...ReviewCardFragment\\n      __typename\\n    }\\n    __typename\\n  }\\n  pageInfo {\\n    prevPageToken\\n    nextPageToken\\n    __typename\\n  }\\n  __typename\\n}\\n\\nfragment ReviewCardFragment on Review {\\n  __typename\\n  id\\n  creator {\\n    ...ReviewerProfileFragment\\n    __typename\\n  }\\n  recommendFor\\n  updatedAt\\n  createdAt\\n  spoilerStatus\\n  lastRevisionAt\\n  text\\n  rating\\n  shelving {\\n    shelf {\\n      name\\n      webUrl\\n      __typename\\n    }\\n    taggings {\\n      tag {\\n        name\\n        webUrl\\n        __typename\\n      }\\n      __typename\\n    }\\n    webUrl\\n    __typename\\n  }\\n  likeCount\\n  viewerHasLiked\\n  commentCount\\n}\\n\\nfragment ReviewerProfileFragment on User {\\n  id: legacyId\\n  imageUrlSquare\\n  isAuthor\\n  ...SocialUserFragment\\n  textReviewsCount\\n  viewerRelationshipStatus {\\n    isBlockedByViewer\\n    __typename\\n  }\\n  name\\n  webUrl\\n  contributor {\\n    id\\n    works {\\n      totalCount\\n      __typename\\n    }\\n    __typename\\n  }\\n  __typename\\n}\\n\\nfragment SocialUserFragment on User {\\n  viewerRelationshipStatus {\\n    isFollowing\\n    isFriend\\n    __typename\\n  }\\n  followersCount\\n  __typename\\n}\\n\"}"
            
    def make_signed_request(self):
        self.check_auth()
        
        auth = BotoAWSRequestsAuth(
            aws_host=self.host,
            aws_region=self.region,
            aws_service=self.service
        )
        response = requests.request(
            self.method,
            url=self.url,
            data=self.reviews_payload,
            auth=auth
        )
        return json.loads(response.text)

In [83]:
# reviews = Reviews(review_num=32)
reviews.make_review_payload(review_num=64)
r = reviews.make_signed_request()
rev = r['data']['getReviews']

In [84]:
len(rev['edges'])

64

In [23]:
r['data']['getReviews'].keys()

dict_keys(['totalCount', 'edges', 'pageInfo', '__typename'])

In [34]:
reviews["edges"][0]["node"]

{'__typename': 'Review',
 'id': 'kca://review:goodreads/amzn1.gr.review:goodreads.v1.CF2yPLr5VvgPgypSEUJVjw',
 'creator': {'id': 731690,
  'imageUrlSquare': 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/users/1288001880i/731690._UX200_CR0,40,200,200_.jpg',
  'isAuthor': False,
  'viewerRelationshipStatus': None,
  'followersCount': 147,
  '__typename': 'User',
  'textReviewsCount': 80,
  'name': 'J',
  'webUrl': 'https://www.goodreads.com/user/show/731690-j',
  'contributor': None},
 'recommendFor': None,
 'updatedAt': 1682566384839.0,
 'createdAt': 1199252675000.0,
 'spoilerStatus': False,
 'lastRevisionAt': 1208687907000.0,
 'text': "<i>Attention: Please ignore the word romance in the goodreads description. I would argue that classification.</i><br /><br />I spent years trying to get anyone I knew to read this book just so I could talk about it with someone other than myself. I've even given it as a gift half a dozen times or so. Useless. They all whine it’s too ha

In [5]:
signer = Signer()
signer.get_signature()


In [21]:
import requests
from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

# self.method = "POST"
# self.uri = 'https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql' 
# # self.uri = "/graphql"
# self.signed_headers = 'accept;content-type;host;x-amz-date;x-amz-security-token'
# self.region = "us-east-1"
# self.service = "appsync" 
# self.host = "kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com"
# self.algorithm = 'AWS4-HMAC-SHA256'
        
auth = BotoAWSRequestsAuth(aws_host="kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com",
                           aws_region="us-east-1",
                           aws_service="appsync")
response = requests.request(
    "POST",
    url="https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql",
#     headers=headers, 
    data=payload,
    auth=auth)
response.text

'{"data":{"getReviews":{"totalCount":1271,"edges":[{"node":{"__typename":"Review","id":"kca://review:goodreads/amzn1.gr.review:goodreads.v1.CF2yPLr5VvgPgypSEUJVjw","creator":{"id":731690,"imageUrlSquare":"https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/users/1288001880i/731690._UX200_CR0,40,200,200_.jpg","isAuthor":false,"viewerRelationshipStatus":null,"followersCount":148,"__typename":"User","textReviewsCount":80,"name":"J","webUrl":"https://www.goodreads.com/user/show/731690-j","contributor":null},"recommendFor":null,"updatedAt":1.682566384839E12,"createdAt":1.199252675E12,"spoilerStatus":false,"lastRevisionAt":1.208687907E12,"text":"<i>Attention: Please ignore the word romance in the goodreads description. I would argue that classification.</i><br /><br />I spent years trying to get anyone I knew to read this book just so I could talk about it with someone other than myself. I\'ve even given it as a gift half a dozen times or so. Useless. They all whine it’s too hard

'{"data":{"getReviews":{"totalCount":1272,"edges":[{"node":{"__typename":"Review","id":"kca://review:goodreads/amzn1.gr.review:goodreads.v1.CF2yPLr5VvgPgypSEUJVjw","creator":{"id":731690,"imageUrlSquare":"https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/users/1288001880i/731690._UX200_CR0,40,200,200_.jpg","isAuthor":false,"viewerRelationshipStatus":null,"followersCount":147,"__typename":"User","textReviewsCount":80,"name":"J","webUrl":"https://www.goodreads.com/user/show/731690-j","contributor":null},"recommendFor":null,"updatedAt":1.682566384839E12,"createdAt":1.199252675E12,"spoilerStatus":false,"lastRevisionAt":1.208687907E12,"text":"<i>Attention: Please ignore the word romance in the goodreads description. I would argue that classification.</i><br /><br />I spent years trying to get anyone I knew to read this book just so I could talk about it with someone other than myself. I\'ve even given it as a gift half a dozen times or so. Useless. They all whine it’s too hard

In [ ]:
payload = "{\"operationName\":\"getReviews\",\"variables\":{\"filters\":{\"resourceType\":\"WORK\",\"resourceId\":\"kca://work/amzn1.gr.work.v1.jY9lyLQNtOx-St1AGeHgBA\"},\"pagination\":{\"limit\":30}},\"query\":\"query getReviews($filters: BookReviewsFilterInput!, $pagination: PaginationInput) {\\n  getReviews(filters: $filters, pagination: $pagination) {\\n    ...BookReviewsFragment\\n    __typename\\n  }\\n}\\n\\nfragment BookReviewsFragment on BookReviewsConnection {\\n  totalCount\\n  edges {\\n    node {\\n      ...ReviewCardFragment\\n      __typename\\n    }\\n    __typename\\n  }\\n  pageInfo {\\n    prevPageToken\\n    nextPageToken\\n    __typename\\n  }\\n  __typename\\n}\\n\\nfragment ReviewCardFragment on Review {\\n  __typename\\n  id\\n  creator {\\n    ...ReviewerProfileFragment\\n    __typename\\n  }\\n  recommendFor\\n  updatedAt\\n  createdAt\\n  spoilerStatus\\n  lastRevisionAt\\n  text\\n  rating\\n  shelving {\\n    shelf {\\n      name\\n      webUrl\\n      __typename\\n    }\\n    taggings {\\n      tag {\\n        name\\n        webUrl\\n        __typename\\n      }\\n      __typename\\n    }\\n    webUrl\\n    __typename\\n  }\\n  likeCount\\n  viewerHasLiked\\n  commentCount\\n}\\n\\nfragment ReviewerProfileFragment on User {\\n  id: legacyId\\n  imageUrlSquare\\n  isAuthor\\n  ...SocialUserFragment\\n  textReviewsCount\\n  viewerRelationshipStatus {\\n    isBlockedByViewer\\n    __typename\\n  }\\n  name\\n  webUrl\\n  contributor {\\n    id\\n    works {\\n      totalCount\\n      __typename\\n    }\\n    __typename\\n  }\\n  __typename\\n}\\n\\nfragment SocialUserFragment on User {\\n  viewerRelationshipStatus {\\n    isFollowing\\n    isFriend\\n    __typename\\n  }\\n  followersCount\\n  __typename\\n}\\n\"}"
headers = {
  'Accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br',
  'Referer': 'https://www.goodreads.com/',
  'content-type': 'application/json; charset=UTF-8',
  'Host': 'kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com',
  'x-amz-date': f"{amz_date}T{amz_time}Z",
  'X-Amz-Security-Token': auth["SessionToken"],
  'Authorization': f'AWS4-HMAC-SHA256 Credential={auth["AccessKeyId"]}/{amz_date}/us-east-1/appsync/aws4_request, SignedHeaders=accept;content-type;host;x-amz-date;x-amz-security-token, Signature={signature}',
  'x-amz-user-agent': 'aws-amplify/2.0.2',
  'Origin': 'https://www.goodreads.com',
  'Connection': 'keep-alive',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'cross-site',
  'TE': 'trailers'
}

response = requests.request("POST", uri, headers=headers, data=payload)
